# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
J. Liu  ->  J. Liu  |  ['J. Liu']
J. Liu  ->  J. Liu  |  ['J. Liu']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
P. Mollière  ->  P. Mollière  |  ['P. Mollière']
T. Henning  ->  T. Henning  |  ['T. Henning']
M. Samland  ->  M. Samland  |  ['M. Samland']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']
K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']
Arxiv has 60 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

  0: tmp_2406.09991/sample631.tex, 353 lines
  1: tmp_2406.09991/fbd_lt.tex, 344 lines


/home/codespace/.python/current/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/home/codespace/.python/current/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: LatexWarning: Found 2 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/codespace/.python/current/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: LatexWarning: Assuming tmp_2406.09991/sample631.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)


L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
P. Mollière  ->  P. Mollière  |  ['P. Mollière']


/home/codespace/.python/current/lib/python3.10/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure empty.eps
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/home/codespace/.python/current/lib/python3.10/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=\hsize]{empty.eps}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/home/codespace/.python/current/lib/python3.10/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure empty.eps
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/home/codespace/.python/current/lib/python3.10/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[angle=-90,width=3cm]{empty.eps}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/home/codespace/.python/current/lib/python3.10/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarni

Found 158 bibliographic references in tmp_2406.10032/aanda.bbl.
Error retrieving bib data for Li24: 'author'
Found 88 bibliographic references in tmp_2406.10217/main.bbl.
syntax error in line 74: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.10032-b31b1b.svg)](https://arxiv.org/abs/2406.10032) | **GASTLI: An open-source coupled interior-atmosphere model to unveil gas giant composition**  |
|| L. Acuña, <mark>L. Kreidberg</mark>, M. Zhai, <mark>P. Mollière</mark> |
|*Appeared on*| *2024-06-17*|
|*Comments*| *18 pages, 9 figures. In review in Astronomy & Astrophysics*|
|**Abstract**|            The metal mass fractions of gas giants are a powerful tool to constrain their formation mechanisms and evolution. The metal content is inferred by comparing mass and radius measurements with interior structure and evolution models. In the midst of the JWST, CHEOPS, TESS, and the forthcoming PLATO era, we are at the brink of obtaining unprecedented precision in radius, age and atmospheric metallicity measurements. To prepare for this wealth of data, we present the GAS gianT modeL for Interiors (GASTLI), an easy-to-use, publicly available Python package. The code is optimized to rapidly calculate mass-radius relations, and radius and luminosity thermal evolution curves for a variety of envelope compositions and core mass fractions. Its applicability spans planets with masses $17 \ M_{\oplus} < M < 6 \ M_{Jup}$, and equilibrium temperatures $T_{eq} < 1000$ K. The interior model is stratified in a core composed of water and rock, and an envelope constituted by H/He and metals (water). The interior is coupled to a grid of self-consistent, cloud-free atmospheric models to determine the atmospheric and boundary interior temperature, as well as the contribution of the atmosphere to the total radius. We successfully validate GASTLI by comparing it to previous work and data of the Solar System's gas giants and Neptune. We also test GASTLI on the Neptune-mass exoplanet HAT-P-26 b, finding a bulk metal mass fraction between 0.60-0.78 and a core mass of 8.5-14.4 $M_{\oplus}$. Finally, we explore the impact of different equations of state and assumptions, such as C/O ratio and transit pressure, in the estimation of bulk metal mass fraction. These differences between interior models entail a change in radius of up to 2.5% for Jupiter-mass planets, but more than 10\% for Neptune-mass. These are equivalent to variations in core mass fraction of 0.07, or 0.10 in envelope metal mass fraction.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.10217-b31b1b.svg)](https://arxiv.org/abs/2406.10217) | **MINDS. A multi-instrument investigation into the molecule-rich JWST-MIRI spectrum of the DF Tau binary system**  |
|| S. L. Grant, et al. -- incl., <mark>T. Henning</mark>, <mark>M. Samland</mark>, <mark>G. Perotti</mark>, <mark>K. Schwarz</mark> |
|*Appeared on*| *2024-06-17*|
|*Comments*| *Submitted to A&A on May 17th, 2024. The reduced and calibrated JWST and ALMA data will become publicly available upon publication*|
|**Abstract**|            Most stars form in multiple systems whose properties can significantly impact circumstellar disk evolution. We investigate the physical and chemical properties of the equal-mass, small separation (~66 mas, ~9 au) DF Tau binary system. Previous observations indicated that only DF Tau A has a circumstellar disk. We present JWST-MIRI MRS observations of DF Tau. The MIRI spectrum shows a forest of H2O lines and emission from CO, C2H2, HCN, CO2, and OH. LTE slab models are used to determine the properties of the gas, and we analyze high angular spatial and spectral resolution data from ALMA, VLTI-GRAVITY, and IRTF-iSHELL to aid in the interpretation of the JWST data. The 1.3 mm ALMA continuum data show two equal-brightness sources of compact (R<3 au) emission, with separations and movement consistent with astrometry from VLTI-GRAVITY and the known orbit. This is interpreted as a robust detection of a disk around DF Tau B, which we suggest may host a small (~1 au) cavity to reconcile all observations. The disk around DF Tau A is expected to be a full disk, and spatially and spectrally resolved dust and gas emission points to hot, close-in (<0.2 au) material. Hot (~500-1000 K) H2O, HCN, and C2H2 emission in the MIRI data likely originate in the DF Tau A disk, while a cold (<200 K) H2O component with an extended emitting area is consistent with an origin from both disks. Despite the very compact outer disks, the inner disk composition and conditions are similar to isolated systems, suggesting that the close binary nature is not a driving factor in setting the inner disk chemistry. However, constraining the geometry of the disks, for instance, via higher resolution ALMA observations, would provide additional insight into the mid-infrared gas emission. JWST observations of spatially resolved binaries will be important for understanding the impact of binarity on inner disk chemistry more generally.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.09813-b31b1b.svg)](https://arxiv.org/abs/2406.09813) | **Diffuse X-ray Explorer: a high-resolution X-ray spectroscopic sky surveyor on the China Space Station**  |
|| H. Jin, et al. -- incl., <mark>J. Li</mark>, <mark>J. Liu</mark> |
|*Appeared on*| *2024-06-17*|
|*Comments*| *12 pages, 6 figures, the full version is published by Journal of Low Temperature Physics*|
|**Abstract**|            DIffuse X-ray Explorer (DIXE) is a proposed high-resolution X-ray spectroscopic sky surveyor on the China Space Station (CSS). DIXE will focus on studying hot baryons in the Milky Way. Galactic hot baryons like the X-ray emitting Milky Way halo and eROSITA bubbles are best observed in the sky survey mode with a large field of view. DIXE will take advantage of the orbital motion of the CSS to scan a large fraction of the sky. High-resolution X-ray spectroscopy, enabled by superconducting microcalorimeters based on the transition-edge sensor (TES) technology, will probe the physical properties (e.g., temperature, density, elemental abundances, kinematics) of the Galactic hot baryons. This will complement the high-resolution imaging data obtained with the eROSITA mission. Here we present the preliminary design of DIXE. The payload consists mainly of a detector assembly and a cryogenic cooling system. The key components of the detector assembly are a microcalorimeter array and frequency-domain multiplexing readout electronics. To provide a working temperature for the detector assembly, the cooling system consists of an adiabatic demagnetization refrigerator and a mechanical cryocooler system.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.09991-b31b1b.svg)](https://arxiv.org/abs/2406.09991) | **On the Interacting/Active Lifetime of Supernova Fallback Disk around Isolated Neutron Stars**  |
|| K. Xu, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2024-06-17*|
|*Comments*| *Accepted in ApJ, comments are welcome*|
|**Abstract**|            The fallback disk model is widely accepted to explain long-period neutron stars (NSs) which can't be simulated by magnetic dipole radiation. However, no confirmed detection of disk was found from the newly discovered long period pulsars GLEAM-X 162759.5-523504.3, GPM J1839-10 and the known slowest isolated NSs 1E 161348-5055. This might be that the disks have either been in noninteracting/inactive state where its emission is too weak to be detected or have been disrupted. In this work, we conduct simulations to examine the lifetime of supernova fallback disks around isolated neutron stars. We assume that the disk's mass varies in a self-similar way and its interaction with the NS occurs only in interacting/active state. Our results reveal that nearly all the interacting lifetimes for the disk are shorter than 0.1 Myr while the existence lifetimes are considerably longer.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2406.10032/./Figures/Saturn_MRrel_comparison_v2.png', 'tmp_2406.10032/./Figures/Neptune_MRrel_comparison_v2.png', 'tmp_2406.10032/./Figures/atmospheric_profiles_Jupiter_v2_referee.png', 'tmp_2406.10032/./Figures/Jupiter_MRrel_comparison_v2_referee.png']
copying  tmp_2406.10032/./Figures/Saturn_MRrel_comparison_v2.png to _build/html/
copying  tmp_2406.10032/./Figures/Neptune_MRrel_comparison_v2.png to _build/html/
copying  tmp_2406.10032/./Figures/atmospheric_profiles_Jupiter_v2_referee.png to _build/html/
copying  tmp_2406.10032/./Figures/Jupiter_MRrel_comparison_v2_referee.png to _build/html/
exported in  _build/html/2406.10032.md
    + _build/html/tmp_2406.10032/./Figures/Saturn_MRrel_comparison_v2.png
    + _build/html/tmp_2406.10032/./Figures/Neptune_MRrel_comparison_v2.png
    + _build/html/tmp_2406.10032/./Figures/atmospheric_profiles_Jupiter_v2_referee.png
    + _build/html/tmp_2406.10032/./Figures/Jupiter_MRrel_comparison_v2_referee.png
found figures ['tmp_2

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# GASTLI

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2406.10032-b31b1b.svg)](https://arxiv.org/abs/2406.10032)<mark>Appeared on: 2024-06-17</mark> -  _18 pages, 9 figures. In review in Astronomy & Astrophysics_

</div>
<div id="authors">

L. Acuña, <mark>L. Kreidberg</mark>, M. Zhai, <mark>P. Mollière</mark>

</div>
<div id="abstract">

**Abstract:** The metal mass fractions of gas giants are a powerful tool to constrain their formation mechanisms and evolution. The metal content is inferred by comparing mass and radius measurements with interior structure and evolution models. In the midst of the JWST, CHEOPS, TESS, and the forthcoming PLATO era, we are at the brink of obtaining unprecedented precision in radius, age and atmospheric metallicity measurements.   To prepare for this wealth of data, we present the GAS gianT modeL for Interiors (GASTLI), an easy-to-use, publicly available Python package. The code is optimized to rapidly calculate mass-radius relations, and radius and luminosity thermal evolution curves for a variety of envelope compositions and core mass fractions. Its applicability spans planets with masses $17  M_{\oplus} < M < 6  M_{Jup}$ , and equilibrium temperatures $T_{eq} < 1000$ K.   The interior model is stratified in a core composed of water and rock, and an envelope constituted by H/He and metals (water). The interior is coupled to a grid of self-consistent, cloud-free atmospheric models to determine the atmospheric and boundary interior temperature, as well as the contribution of the atmosphere to the total radius.   We successfully validate GASTLI by comparing it to previous work and data of the Solar System’s gas giants and Neptune. We also test GASTLI on the Neptune-mass exoplanet HAT-P-26 b, finding a bulk metal mass fraction between 0.60-0.78 and a core mass of 8.5-14.4 $M_{\oplus}$ . Finally, we explore the impact of different equations of state and assumptions, such as C/O ratio and transit pressure, in the estimation of bulk metal mass fraction. These differences between interior models entail a change in radius of up to 2.5 \% for Jupiter-mass planets, but more than 10 \% for Neptune-mass. These are equivalent to variations in core mass fraction of 0.07, or 0.10 in envelope metal mass fraction.

</div>

<div id="div_fig1">

<img src="tmp_2406.10032/./Figures/Saturn_MRrel_comparison_v2.png" alt="Fig10.1" width="50%"/><img src="tmp_2406.10032/./Figures/Neptune_MRrel_comparison_v2.png" alt="Fig10.2" width="50%"/>

**Figure 10. -** Left: Mass-radius relations for Saturn. GASTLI's fiducial model for Saturn can reproduce with a better precision Saturn's mass and radius data than HG21. We assumed an equilibrium temperature of $T_{eq} = $ 100 K across all models, while we adopt an internal temperature $T_{int}$ = 77 for Saturn. Models for Saturn were calculated for a fixed envelope metallicity of 8.9 $\times$ solar, equivalent to $Z_{env}$ = 0.10 for the HG21 model. Right: Mass-radius relations for Neptune. The fiducial model (black) agrees well with mass and radius data. We assumed an internal temperature $T_{int}$ = 52 K for Neptune. (*fig:Saturn_and_Neptune*)

</div>
<div id="div_fig2">

<img src="tmp_2406.10032/./Figures/atmospheric_profiles_Jupiter_v2_referee.png" alt="Fig5" width="100%"/>

**Figure 5. -** Atmospheric profiles for Jupiter's interior-atmosphere models. Our clear petitCODE fiducial model is 50 K warmer than Jupiter's atmospheric data provided by the Voyager and the Galileo probe  ([Seiff, Kirk and Knight 1998](), [Gupta, Atreya and Steffes 2022](), Li24) . We assumed the fiducial case, with an internal temperature of $T_{int} $ = 107 K, a solar composition, and a core mass fraction of CMF = 0.03. For the [ and Guillot (2010)]() atmospheric models, we adopted an infrared opacity $\kappa_{th}$ = 0.01 cm$^{2}$/g, and a visible-to-thermal opacity ratio of $\kappa_{v}/\kappa_{th} = \gamma$ = 0.4. (*fig:Jupiter_atm_prof*)

</div>
<div id="div_fig3">

<img src="tmp_2406.10032/./Figures/Jupiter_MRrel_comparison_v2_referee.png" alt="Fig9" width="100%"/>

**Figure 9. -** Left: GASTLI models for Jupiter. GASTLI can reproduce Jupiter's mass and radius data with the fiducial model within 1.6\%. Solid lines show the total radius obtained with GASTLI for core mass fractions (CMF) equal to 0 and 0.03 (fiducial) assuming a solar metallicity in the envelope. The global average equilibrium temperature (122 K) and internal temperature (107 K) of Jupiter were adopted .
      Right: Comparison of mass-radius relations between GASTLI and two widely used interior models for gas giants: [Fortney, Marley and Barnes (2007)]()(F07) and [ and Müller (2021)]()(MH21). For the same composition, GASTLI agrees within uncertainties with other publicly available mass-radius relations. The fiducial case at CMF = 0.03 is indicated in black for all models. F07 and MH21 models were obtained assuming the irradiation and age of Jupiter. For MH21 at CMF = 0, a solar $Z_{env}$ = 0.013 is not available, so $Z_{env}$ = 0 is assumed instead. (*fig:Jupiter_MRrel*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2406.10032"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]{\unskip\protect\href{https://orcid.org/#1}{\protect\includegraphics[width=8pt,clip]{logo_orcid}}}$
$\newcommand{\Lsun}{L_{\odot}}$
$\newcommand{\Msun}{M_{\odot}}$
$\newcommand{\Mearth}{M_{\oplus}}$
$\newcommand{\Rsun}{R_{\odot}}$
$\newcommand{\Mdot}{\dot{M}}$
$\newcommand{\Mdust}{{M_{\rm{dust}}}}$
$\newcommand{\Mdisk}{{M_{\rm{disk}}}}$
$\newcommand{\Av}{A_V}$
$\newcommand{\msunyr}{\rm{M_{\sun}   yr^{-1}}}$
$\newcommand{\Teff}{T_{\rm eff}}$
$\newcommand{\mic}{\mum}$
$\newcommand{\tstar}{T_{*}}$
$\newcommand{\rstar}{R_{*}}$
$\newcommand{\mstar}{M_{*}}$
$\newcommand{\rdisk}{r_{\rm{disk}}}$
$\newcommand{\hwall}{h_{\rm{wall}}}$
$\newcommand{\amaxbig}{a_{\rm{max,big}}}$
$\newcommand{\amaxsmall}{a_{\rm{max,small}}}$
$\newcommand{\brgamma}{Br\gamma}$
$\newcommand{\Lacc}{L_{\rm{acc}}}$
$\newcommand{\LaccL}{L_{\rm{acc}}/L_{*}}$
$\newcommand{\Lbrg}{L_{\rm{Br\gamma}}}$
$\newcommand{\Fbrg}{F_{\rm{Br\gamma}}}$
$\newcommand{\angstrom}{\mbox{\normalfontÅ}}$
$\newcommand{\Lstar}{L_{*}}$
$\newcommand{\LNIR}{L_{\rm{NIR}}}$
$\newcommand{\LIR}{L_{\rm{IR}}}$
$\newcommand{\vsini}{vsin(i)}$
$\newcommand{\Ri}{R_{i}}$
$\newcommand{\mdotmdisk}{\dot{M}--M_{\rm{disk}}}$
$\newcommand{\tdisk}{t_{\rm{disk}}}$
$\newcommand{\nk}[1]{\textcolor{teal}{Nico}: #1}$
$\newcommand{\sg}[1]{\textcolor{red}{Sierra}: #1}$</div>



<div id="title">

# MINDS. A multi-instrument investigation into the molecule-rich JWST-MIRI spectrum of the DF Tau binary system

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2406.10217-b31b1b.svg)](https://arxiv.org/abs/2406.10217)<mark>Appeared on: 2024-06-17</mark> -  _Submitted to A&A on May 17th, 2024. The reduced and calibrated JWST and ALMA data will become publicly available upon publication_

</div>
<div id="authors">

S. L. Grant, et al. -- incl., <mark>T. Henning</mark>, <mark>M. Samland</mark>, <mark>G. Perotti</mark>, <mark>K. Schwarz</mark>

</div>
<div id="abstract">

**Abstract:** The majority of young stars form in multiple systems, the properties of which can significantly impact the evolution of any circumstellar disks. We investigate the physical and chemical properties of the equal-mass, small separation ( $\sim$ 66 milliarcsecond, $\sim$ 9 au) binary system DF Tau. Previous spatially resolved observations indicate that only DF Tau A has a circumstellar disk, while DF Tau B does not, as concluded by a lack of accretion signatures and near-infrared excess. We present JWST-MIRI MRS observations of DF Tau. The MIRI spectrum shows emission from a forest of $H_2$ O lines and emission from CO, $C_2$ $H_2$ , HCN, $CO_2$ , and OH. Localthermodynamic equilibrium slab models are used to determine the properties of the gas. The binary system is not spatially or spectrally resolved in the MIRI observations, therefore, we analyze high angular spatial and spectral resolution observations from ALMA, VLTI-GRAVITY, and IRTF-iSHELL to aid in the interpretation of the molecular emission observed with JWST. The 1.3 mm ALMA observations show two equal brightness sources of compact ( $R\lesssim$ 3 au) continuum emission that are detected at high significance, with separations consistent with astrometry from VLTI-GRAVITY and movement consistent with the known orbital parameters of the system. This is interpreted as a robust detection of the disk around DF Tau B, which we suggest may host a small ( $\sim$ 1 au) cavity to reconcile all of the observations of this source. In contrast, the disk around DF Tau A is expected to be a full disk, and spatially and spectrally resolved dust and gas emission traced by ground-based infrared observations point to hot, close-in ( $\lesssim0.2$ au) material around this star.High temperature ( $\sim$ 500-1000 K) emission from $H_2$ O, HCN, and potentially $C_2$ $H_2$ in the MIRI data likely originates in the disk around DF Tau A, while a cold ( $\lesssim$ 200 K) $H_2$ O component with an extended emitting area is consistent with an origin from both disks. Given the unique characteristics of this binary pair, complementary observations are critical for constraining the properties of these disks. Despite the very compact outer disk properties, the inner disk composition and conditions of the DF Tau disks are remarkably similar to isolated systems, suggesting that neither the outer disk evolution nor the close binary nature are driving factors in setting the inner disk chemistry in this system. However, constraining the geometry of the disk around DF Tau B, via higher angular resolution ALMA observations, for instance, would provide additional insight into the properties of the mid-infrared gas emission observed with MIRI. JWST observations of spatially resolved binaries, at a range of separations, will be important for understanding the impact of binarity on inner disk chemistry more generally.

</div>

<div id="div_fig1">

<img src="tmp_2406.10217/./DFTau_miri_corner_examples.png" alt="Fig11" width="100%"/>

**Figure 11. -** The posterior distributions of the MCMC modeling of the JWST-MIRI spectra. The column densities, $N$, are in log$_{10}$ space. Examples of the correlations seen in optically thick and optically thin cases are shown in the upper right.  (*fig: corner plot*)

</div>
<div id="div_fig2">

<img src="tmp_2406.10217/./GRAVITY.png" alt="Fig10" width="100%"/>

**Figure 10. -** The VLTI-GRAVITY visibilities squared (top) and the closure phases (bottom) in the observations (colors) compared to the best-fit model (red). (*fig: gravity model*)

</div>
<div id="div_fig3">

<img src="tmp_2406.10217/./DFTau_lineprofiles.png" alt="Fig1" width="100%"/>

**Figure 1. -** The stacked line profiles for CO v=1-0 (black), CO v=2-1 (gray), and the 5 $\mic$ $H_2$O (blue) lines from IRFT-iSHELL spectrum of DF Tau \citep{banzatti23a}. A Keplerian line profile, calculated using an inclination of 67$^\circ$ and a stellar mass of 0.55 $\Msun$, is shown in red, which matches the v=2-1 lines.   (*fig: line profiles*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2406.10217"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

98  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
